# Analyze TIES Hyperparameter Gridsearch

## Imports

In [1]:
import os
import json
import pandas as pd

# Iterate Through Each Hyperparameter Combination folder

In [7]:
unlearn_method = "esd"
training_method = "independent"
merge_method = "ties"
concept_type = "style"
concepts = ["Cartoon"]
base_dir = f"/fs/scratch/PAS2099/lee.10369/CUIG/{unlearn_method}/eval_results/{training_method}/merge/{merge_method}/{concept_type}"
all_results = []

# Find hypeparameter values
print(f"Searching in {base_dir}")
combos = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
print(f"Found {len(combos)} combinations in {base_dir}")

# Iterate through each combination
for combo in combos:
    combo_dir = os.path.join(base_dir, combo)
    for concept in concepts:
        # Each concept folder contains a "metrics" folder with the JSON files
        metrics_dir = os.path.join(combo_dir, f"thru{concept}", 'metrics')
        summary_file = os.path.join(metrics_dir, 'summary.json')
        
        if os.path.exists(summary_file):
            with open(summary_file, 'r') as f:
                summary_data = json.load(f)

            unlearn_accuracy_avg = summary_data.get('UA', {})
            retention_accuracy = summary_data.get('IRA', {})
            cross_retention_accuracy = summary_data.get('CRA', {})
            final_score = (unlearn_accuracy_avg + (retention_accuracy + cross_retention_accuracy)/2) / 2
            
            all_results.append({
                'combo': combo,
                'concept': concept,
                'concept_type': concept_type,
                'UA': unlearn_accuracy_avg,
                'IRA': retention_accuracy,
                'CRA': cross_retention_accuracy,
                'final_score': final_score
            })
        else:
            print(f"Missing metrics for combo {combo} and concept {concept} in {combo}")

# Combine results into a DataFrame for further analysis
df_results = pd.DataFrame(all_results)

# ---------------------------
# 2. Extract Hyperparameter Values from 'combo'
# ---------------------------
# The combo string is in the format "2Rank1Alpha0.1Dropout"
if merge_method == "ties":
    pattern = r'lambda(?P<lambda>[\d\.]+)_topk(?P<topk>[\d\.]+)'
extracted = df_results['combo'].str.extract(pattern)
df_results['topk'] = extracted['topk'].astype(float)
df_results['lambda'] = extracted['lambda'].astype(float)

df_results.sort_values(['final_score', 'UA'], ascending=False, inplace=True)

df_save_dir = f"/users/PAS2099/justinhylee135/Research/UnlearningDM/CUIG/Analysis/Notebooks/output/{unlearn_method}/{training_method}/merge/{merge_method}/{concept_type}"
os.makedirs(df_save_dir, exist_ok=True)
df_results.to_excel(os.path.join(df_save_dir, f"thru{concepts[0]}.xlsx"), index=False)

print(f"Total number of combinations: {len(df_results)}")
df_results.head(len(df_results))

Searching in /fs/scratch/PAS2099/lee.10369/CUIG/esd/eval_results/independent/merge/ties/style
Found 17 combinations in /fs/scratch/PAS2099/lee.10369/CUIG/esd/eval_results/independent/merge/ties/style
Total number of combinations: 17


,combo,concept,concept_type,UA,IRA,CRA,final_score,topk,lambda
14,lambda1.25_topk0.20,Cartoon,style,97.50,66.67,98.50,90.0425,0.2,1.25
1,lambda1.25_topk0.80,Cartoon,style,98.33,61.67,97.84,89.0425,0.8,1.25
16,lambda1.25_topk0.60,Cartoon,style,100.00,54.58,97.67,88.0625,0.6,1.25
15,lambda1.25_topk0.40,Cartoon,style,100.00,51.04,97.50,87.1350,0.4,1.25
10,lambda1.00_topk0.30,Cartoon,style,84.17,80.42,98.33,86.7725,0.3,1.00
7,lambda1.50_topk0.20,Cartoon,style,100.00,39.79,96.50,84.0725,0.2,1.50
9,lambda1.50_topk0.80,Cartoon,style,99.17,38.75,96.00,83.2725,0.8,1.50
12,lambda1.50_topk0.60,Cartoon,style,100.00,29.38,94.67,81.0125,0.6,1.50
11,lambda1.50_topk0.40,Cartoon,style,100.00,27.08,95.17,80.5625,0.4,1.50
4,lambda1.75_topk0.20,Cartoon,style,100.00,25.42,93.84,79.8150,0.2,1.75
